In [1]:
import os, sys

import numpy as np

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#from ddgclib._catenoid import *
from ddgclib._curvaturesplay import *
import copy

from timeit import default_timer as timer

In [2]:
'''
Input-Parameter: P, U, V,T
'''
P = np.array([0.0, 0.0, -1.0]) # Stützpunkte = (0,0, boundarybottom.vx[2])
U = np.array([1.5430806348152437, -3.779457520505508e-16, 1.0]) # (boundary_bottom.vx[0], boundary_bottom.vx[1], boundary_bottom.vx[2])
V = np.array([1.5430806348152437, 0.0, -1.0]) # (boundary_top.vx[0], boundary_top.vx[1], boundary_top.vx[2])
N = np.cross(U, V)  # Normalenvektor berechnen
T = np.array([1.29468334, 0, 0.75])  #

def locate_in_plane(P, U, V, T):
    N = np.cross(U, V)
    # Überprüfe, ob der Punkt in der Ebene liegt
    return np.isclose(np.dot(N, T - P), 0)

print(f'liegt in der Normalform dummy x: {locate_in_plane(P, U, V, T)}')

liegt in der Normalform dummy x: True


In [3]:
def catenoid_N(r, length, refinement=2):

    v_l = -length/2
    v_u = length/2
    a = r

    def sech(x):
        return 1 / np.cosh(x)

    def catenoid(u, v):
        x = a * np.cos(u) * np.cosh(v / a)
        y = a * np.sin(u) * np.cosh(v / a)
        z = v
        return x, y, z

    # Equation: x**2/a**2 + y**2/b**2 + z**2/c**2 = 1
    # TODO: Assume that a > b > c?
    # Exact values:
    R = a
    domain = [#(-2.0, 2.0),  # u
              (0.0, 2 * np.pi),  # u
              (v_l, v_u)  # v
              ]
    HC_plane = Complex(2, domain)
    HC_plane.triangulate()
    for i in range(refinement):
        HC_plane.refine_all()

    # H
    HC = Complex(3, domain)
    bV = set()
    cdist = 1e-8

    u_list = []
    v_list = []
    for v in HC_plane.V:
        #print(f'-')
        #print(f'v.x = {v.x}')
        x, y, z = catenoid(*v.x_a)
        #print(f'tuple(x, y, z) = {tuple([x, y, z])}')
        v2 = HC.V[tuple([x, y, z])]

        u_list.append(v.x_a[0])
        v_list.append(v.x_a[1])

        #TODO: Does not work at all:
        boundary_bool = (
                        v.x[1] == domain[1][0] or v.x[1] == domain[1][1]
                        #v.x[2] == domain[0][0] or v.x[2] == domain[0][1]
                       # or v.x[1] == domain[1][0] or v.x[1] == domain[1][1]
                         )
        if boundary_bool:
            bV.add(v2)

    # Connect neighbours
    for v in HC_plane.V:
        for vn in v.nn:
            v1 = list(HC.V)[v.index]
            v2 = list(HC.V)[vn.index]
            v1.connect(v2)


    # Remerge:
    HC.V.merge_all(cdist=cdist)
    bVc = copy.copy(bV)
    for v in bVc:
        #print(f'bv = {v.x}')
        if not (v in HC.V):
            bV.remove(v)


    H_f = []
    bV_H_f = []
    K_f = []
    bV_K_f = []

    neck_verts = []
    neck_sols = []
    for vert in HC.V:
        if not (vert in bV):
            for u_i, v_i in zip(u_list, v_list):
                x, y, z = catenoid(u_i, v_i)
                va = np.array([x, y, z])
                #print(f'va == vert.x_a = {va == vert.x_a}')
                ba = va == vert.x_a
                if ba.all():
                    #print(f'ba.all() = {ba.all()}')
                    u = u_i
                    v = v_i
                    #nom = c * (a ** 2 * (u ** 2 - 1) + c ** 2 * (u ** 2 + 1))
                    #denom = 2 * a * (u ** 2 * (a ** 2 + c ** 2) + c ** 2) ** (3 / 2.0)
                    H_f_i = 0.0
                    #K_f_i = - c ** 2 / ((u ** 2 * (a ** 2 + c ** 2) + c ** 2) ** 2)
                    #K_f_i = -(np.sech(v/a))**4 / (a**2)
                    #K_f_i = -(math.sech(v/a))**4 / (a**2)
                    K_f_i = -(sech(v/a))**4 / (a**2)
                    H_f.append(H_f_i)
                    K_f.append(K_f_i)
                    #TODO: MERGE NEAR VERTICES

                    if z == 0.0:
                        neck_verts.append(vert.index)
                        neck_sols.append((H_f_i, K_f_i))


    return HC, bV, K_f, H_f, neck_verts, neck_sols

In [4]:
r = 1
length = 2

HC, bV, K_f, H_f, neck_verts, neck_sols = catenoid_N(r, length, refinement=3)

In [5]:
from ddgclib._plotting import *
plot_polyscope(HC)

np.zeros([self.V.size(), self.dim]) = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]

In [38]:
number_v = 0
for v in HC.V:
    number_v += 1
    #print(number_v)

boundary_bottom = []
boundary_top    = []

v_l = - length/2
v_u =   length/2

for v in HC.V:
   # print(v.index)
    if v.x[2] == v_l:
        boundary_bottom.append(v)

    if v.x[2] == v_u:
        boundary_top.append(v)

        #print(v.cache[1])

#print(HC.V.cache)

'''
v.index=1
v.x = (1.5430806348152437, -3.779457520505508e-16, 1.0)
'''

'\nv.index=1\nv.x = (1.5430806348152437, -3.779457520505508e-16, 1.0)\n'

In [39]:
boundary_bottom_x = []
for v in boundary_bottom:
    #dummy_item.append(v.index)
    boundary_bottom_x.append(v.x)

boundary_top_x = []
for v in boundary_top:
    #dummy_item.append(v.index)
    boundary_top_x.append(v.x)

In [40]:
#print(boundary_bottom[].index)

In [41]:

# Gemeinsame Elemente finden, bei denen die ersten beiden Werte identisch sind
gemeinsame_elemente = [tupel for tupel in boundary_top_x if any((tupel[0] == other[0]) and (tupel[1] == other[1]) for other in boundary_bottom_x)] # is a list

# Ausgabe der gemeinsamen Elemente
print("Gemeinsame Elemente mit identischen ersten beiden Werten:")
for element in gemeinsame_elemente:
    print(element)

print('-----')
print(type(gemeinsame_elemente))
# Abfrage der einzelnen Elemente gemeinsame_elemente[i][0]


Gemeinsame Elemente mit identischen ersten beiden Werten:
(-1.5430806348152437, 1.889728760252754e-16, 1.0)
(9.44864380126377e-17, 1.5430806348152437, 1.0)
(-2.834593140379131e-16, -1.5430806348152437, 1.0)
(1.0911227807955015, 1.0911227807955015, 1.0)
(-1.0911227807955013, 1.0911227807955015, 1.0)
(-1.0911227807955017, -1.0911227807955013, 1.0)
(1.091122780795501, -1.0911227807955017, 1.0)
-----
<class 'list'>


In [45]:
type(gemeinsame_elemente[0][0])

numpy.float64

In [10]:
vlist = []
for v in HC.V:
    if (abs(v.x[0]) <= 1e-8) and (0.5 <= v.x[1] <= 2):
        vlist.append(v)

vlist
for v in vlist:
    print(v.x)

(6.904717644627194e-17, 1.1276259652063807, 0.5)
(6.904717644627194e-17, 1.1276259652063807, -0.5)
(6.123233995736766e-17, 1.0, 0.0)
(9.44864380126377e-17, 1.5430806348152437, 1.0)
(9.44864380126377e-17, 1.5430806348152437, -1.0)
(7.927648702445397e-17, 1.2946832846768448, 0.75)
(6.315583756830844e-17, 1.0314130998795732, 0.25)
(6.315583756830844e-17, 1.0314130998795732, -0.25)
(7.927648702445397e-17, 1.2946832846768448, -0.75)


In [12]:
print(len(vlist))

9


In [48]:
''''
P = np.array([0.0, 0.0, -1.0]) # Stützpunkte = (0,0, boundarybottom.vx[2])
U = np.array([1.5430806348152437, -3.779457520505508e-16, 1.0]) # (boundary_bottom.vx[0], boundary_bottom.vx[1], boundary_bottom.vx[2])
V = np.array([1.5430806348152437, 0.0, -1.0]) # (boundary_top.vx[0], boundary_top.vx[1], boundary_top.vx[2])
N = np.cross(U, V)  # Normalenvektor berechnen
T = np.array([1.29468334, 0, 0.75])  #
'''

iteration = 0
for i in range(len(gemeinsame_elemente)):
    iteration += 1
    P = np.array([0.0, 0.0, -gemeinsame_elemente[i][2]])
    U = np.array([gemeinsame_elemente[i][0], gemeinsame_elemente[i][1]], -gemeinsame_elemente[i][2])
    V = np.array([gemeinsame_elemente[i][0], gemeinsame_elemente[i][1]], gemeinsame_elemente[i][2])
    T = np.array([1.29468334, 0, 0.75])
    print(f' U = {U}')
    print(f'P')
    print(f'liegt in der Normalform : {locate_in_plane(P, U, V, T)}')

 U = [-1.54308063e+00  1.88972876e-16]
P
liegt in der Normalform : [ True  True  True]
 U = [9.44864380e-17 1.54308063e+00]
P
liegt in der Normalform : [ True  True  True]
 U = [-2.83459314e-16 -1.54308063e+00]
P
liegt in der Normalform : [ True  True  True]
 U = [1.09112278 1.09112278]
P
liegt in der Normalform : [ True  True  True]
 U = [-1.09112278  1.09112278]
P
liegt in der Normalform : [ True  True  True]
 U = [-1.09112278 -1.09112278]
P
liegt in der Normalform : [ True  True  True]
 U = [ 1.09112278 -1.09112278]
P
liegt in der Normalform : [ True  True  True]


In [100]:


dummy = 0
for v in boundary_bottom:
    dummy +=1
    print(v.x)

    for vnn in v.nn:
        if vnn in boundary_bottom:
            continue
        else:
            pass
          #  print(vnn.x)



(1.5430806348152437, 0.0, -1.0)
(-1.5430806348152437, 1.889728760252754e-16, -1.0)
(-2.834593140379131e-16, -1.5430806348152437, -1.0)
(9.44864380126377e-17, 1.5430806348152437, -1.0)
(1.091122780795501, -1.0911227807955017, -1.0)
(-1.0911227807955017, -1.0911227807955013, -1.0)
(1.0911227807955015, 1.0911227807955015, -1.0)
(-1.0911227807955013, 1.0911227807955015, -1.0)


Die allgemeine Parameterform einer Ebene im dreidimensionalen Raum lautet:

r=P+s⋅U+t⋅Vr=P+s⋅U+t⋅V

Hierbei sind:

    r der Punkt auf der Ebene.
    P ein Punkt auf der Ebene (ein sogenannter Stützpunkt).
    U und VV zwei linear unabhängige Richtungsvektoren, die in der Ebene liegen.
    s und tt sind Parameter, die alle reellen Zahlen annehmen können.

Diese Form ermöglicht es, jeden Punkt rr in der Ebene durch verschiedene Kombinationen von ss und tt zu repräsentieren. Ändert man die Werte von ss und tt, so durchläuft man verschiedene Punkte auf der Ebene.

Die Normalenform einer Ebene lautet:

N⋅(r−P)=0N⋅(r−P)=0

Hierbei sind:

    NN der Normalenvektor der Ebene.
    PP ein Punkt auf der Ebene.
    ⋅⋅ das Skalarprodukt.

Diese Form ermöglicht es, zu überprüfen, ob ein bestimmter Punkt rr auf der Ebene liegt, indem man den Normalenvektor NN mit dem Vektor r−Pr−P skalarweise multipliziert und prüft, ob das Ergebnis gleich null ist.

In [37]:
import numpy as np

# Funktion zur Überprüfung, ob ein Punkt in der Ebene liegt (Parameterform)
def liegt_in_ebene_parametrisch(P, U, V, T):
    # Versuche, s und t zu berechnen
    try:
        s, t = np.linalg.solve(np.array([U, V]).T, T - P)
        return True
    except np.linalg.LinAlgError:
        return False

# Funktion zur Überprüfung, ob ein Punkt in der Ebene liegt (Normalenform)
def liegt_in_ebene_normalenform(P, N, T):
    # Überprüfe, ob der Punkt in der Ebene liegt
    return np.isclose(np.dot(N, T - P), 0)

'''
E: x = [0,0,boundary_bottom

'''
# Beispieldaten


Liegt T in der Ebene? (Parameterform): False
Liegt T in der Ebene? (Normalenform): True
liegt in der Normalform dummy x: True
